In [1]:
import sources.logins as lg
from time import sleep
import warnings
warnings.filterwarnings('ignore')

def login_exfm(driver_on = True):
    # generate key using fernet
    # key = Fernet.generate_key().decode()
    key = 'cLAiAdZU1U0sxWWK8sxhF0IWIBP4KrR3xhdu3x1EDQI='
    
    
    # login ExFM and download latest incompleted data
    name,pw = lg.save_id('exfm.txt',key)
    driver = lg.login_url('exfm',name,pw)
    
    
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search	
            break
        except Exception as e:
            print ('Wrong Username or Password. Try again...')
            name,pw = lg.save_id('exfm.txt',key)
            driver = lg.login_url('exfm',name,pw)
    
    d_type_menu = [
                    'Incompleted',
                    'History',
                    'Equipments',
                    'Customers',
                    'No Download'
                    
                    ]
    #border table
    print(f'\n{"_"*50}')
    print(f'{"|  No.|  Function": <49}|')
    print(f'|{"_"*48}|')
    for i in range(1,1+len(d_type_menu)):
        print(f'|{i: >3}  |  {d_type_menu[i-1]: <40}|')
    print(f'|{"_"*48}|') #bottom border
    while True:
        ind = str(input('Select Dataset to download: '))
        try:
            ind = int(ind)
        except:
            if ind.upper() == 'Q' or ind.upper() == 'QUIT':
                break
            else:
                print('Only accept number')
                continue
        if 0<ind<=len(d_type_menu):
            d_type = d_type_menu[ind-1]
            break
        else:
            print(f'Input number must be less than {len(d_type_menu)}\n')
    
    print(d_type)
    
    # driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search
    
    if d_type == d_type_menu[0]: # Incompleted
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[1]: # History
        Select(driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_STATUS"]')).select_by_index(0)
        print('Select Status "All" and download.')
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[2]: # Equipments
        driver.find_element_by_xpath('//*[@id="EQP_MGT_LINK"]').click() # Equipments
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    if d_type == d_type_menu[3]: # Customers
        driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click() 
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    else:
        print('No Download')

    # check latest file
    i_wait = 0
    while True:
        try: # check file already
            file, ctime = lg.file_latest(folder_name='Downloads')
            if file.endswith('xls'):
                print (f'\nDonwload file {file} succesful at {ctime}')
                break
            else:
                print(f'Please wait for compressing file {file}...{i_wait}s')
                i_wait +=3
                sleep(3)
        except Exception as e: #waiting for download
            print(f'Please wait for dowloading...{i_wait}s')
            i_wait +=3
            sleep(3)
            if d_type == d_type_menu[-1]: break
    if driver_on:
        return driver
    else:
        #finish close driver
        driver.close()
    sleep(2)

def auto_accept(driver):
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    driver.switch_to.alert.accept()
    
def export_all_translog_in_page(driver):
    for i in range(50):
        # translog button
        driver.find_element_by_xpath('//*[@id="sidTRANSLOG_BUTTON_IMAGE"]').click()
        p = driver.current_window_handle
        chwd = driver.window_handles
        for w in chwd:
            if(w!=p):
                driver.switch_to.window(w)
                break
        sn = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table/tbody/tr[1]/td[1]').get_attribute('innerHTML')
        rma = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table/tbody/tr[1]/td[2]').get_attribute('innerHTML')
        print(i+1,sn,rma)
        # click export
        driver.find_element_by_xpath('//*[@id="export"]').click()
        driver.close()
        sleep(0.5)
        driver.switch_to.window(p)

        # next button
        try:
            next_button = driver.find_element_by_xpath('//*[@id="sidSHOW_NEXT_REPAIR"]')
            next_button.click()
        except Exception as e:
            print(f"Finish all {i+1} jobs in this page.\n")
            break

In [81]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import math

In [36]:

driver = login_exfm()


Select account:
1   |   thongle
2   |   Nguyen
0   |   CREATE NEW ACCOUNT
Select account by number: 1
thongle is selected to login
Removed Downloads


Create new folder: Downloads
Select Chrome Driver and press enter for authorize Certificate...

Typing your ID...
Typing your password...
Login sucessful!


__________________________________________________
|  No.|  Function                                |
|________________________________________________|
|  1  |  Incompleted                             |
|  2  |  History                                 |
|  3  |  Equipments                              |
|  4  |  Customers                               |
|  5  |  No Download                             |
|________________________________________________|
Select Dataset to download: 5
No Download
No Download
Please wait for dowloading...0s


In [94]:
#RMA search tab
driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click()

In [95]:
#show all
driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
sleep(0.5)
ttl = driver.find_element_by_xpath('//*[@id="TH_search_result"]/thead/tr[1]/th/div/span').get_attribute('innerHTML')
ttl = int(ttl.split('/')[1][:-1])
print(ttl)

230


In [71]:
driver.switch_to.window(p)

In [96]:
pg = math.ceil(ttl/50)
for i in range(pg):
    first_row = i * 50
    #Fisrt row
    driver.find_element_by_xpath(f'//*[@id="search_result[{first_row}].sidRS_REPAIR_ID_data"]').click()
    # export translog
    print(f'Export for page {i + 1}')
    export_all_translog_in_page(driver)
    # return
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
    sleep(0.5)
    # next page
    driver.find_element_by_xpath('//*[@id="_eventcursornextpage_search_result"]').click()
    sleep(0.5)

Export for page 6
1 2C730K027 FMSV2023020019
2 2C730K032 FMSV2023020022
3 1C728K180 FMSV2023020034
4 2V567K684 FMSV2023020067
5 3B087A006 FMSV2023030008
6 3B063A049 FMSV2023030009
7 2V564K581 FMSV2023030032
8 1C728K450 FMSV2023030040
9 4G361A481 FMSV2023030041
10 2V644K028 FMSV2023030060
11 1C741K218 FMSV2023030069
12 1C603K041 FMSV2023030075
13 2V567K740 FMSV2023030079
14 1C675K021 FMSV2023040004
15 7G391K201 FMSV2023040010
16 4G374K157 FMSV2023040017
17 6G402K258 FMSV2023040024
18 5C728K054 FMSV2023040025
19 1C692K648 FMSV2023040038
20 8G391K423 FMSV2023040049
21 3Y196K006 FMSV2023040060
22 2G348K408 FMSV2023040070
23 7G391K085 FMSV2023040077
24 8G391K374 FMSV2023040090
25 8G391K428 FMSV2023040091
26 1C643K395 FMSV2023040092
27 SG202A469 FMSV2023050018
28 1C666K565 FMSV2023050021
29 3G202K117 FMSV2023050024
30 1G402K303 FMSV2023050025
31 4G374K154 FMSV2023050033
32 3C309K012 FMSV2023050034
33 3c309k028 FMSV2023050035
34 2V567K886 FMSV2023050042
35 4C591A001 FMSV2023050045L
36 1B083K2

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [97]:
# driver.find_element_by_xpath()

### Read excel file

In [99]:
import os
os.listdir('Downloads')

['export (1).csv',
 'export (10).csv',
 'export (100).csv',
 'export (11).csv',
 'export (12).csv',
 'export (13).csv',
 'export (14).csv',
 'export (15).csv',
 'export (16).csv',
 'export (17).csv',
 'export (18).csv',
 'export (19).csv',
 'export (2).csv',
 'export (20).csv',
 'export (21).csv',
 'export (22).csv',
 'export (23).csv',
 'export (24).csv',
 'export (25).csv',
 'export (26).csv',
 'export (27).csv',
 'export (28).csv',
 'export (29).csv',
 'export (3).csv',
 'export (30).csv',
 'export (31).csv',
 'export (32).csv',
 'export (33).csv',
 'export (34).csv',
 'export (35).csv',
 'export (36).csv',
 'export (37).csv',
 'export (38).csv',
 'export (39).csv',
 'export (4).csv',
 'export (40).csv',
 'export (41).csv',
 'export (42).csv',
 'export (43).csv',
 'export (44).csv',
 'export (45).csv',
 'export (46).csv',
 'export (47).csv',
 'export (48).csv',
 'export (49).csv',
 'export (5).csv',
 'export (50).csv',
 'export (51).csv',
 'export (52).csv',
 'export (53).csv',
 'ex

In [172]:
# read file to pandas
import pandas as pd
from datetime import datetime as dt
file = 'Downloads/export (99).csv'
df = pd.read_csv(file,encoding = 'utf-8')
df

,S/N,RMA#,Time,User,Station,Screen,Type,Updated Item(After),Updated Item(Before)
0,2G361K131,FMSV2023070056,2023/07/21 11:09:26,Nguyen,Inspection,Parts Selection,Save,"[0][Appr.:1, Parts No.:""370Y200577A"", Delete:0...",NaN
1,2G361K131,FMSV2023070056,2023/07/21 11:05:44,Nguyen,Receive,Inspection,Completion,"Status:""Inspection"", Repair Type:""Major"", Insp...","Status:""Receive"", Repair Type:"""", Inspection:""..."
2,2G361K131,FMSV2023070056,2023/07/21 10:58:51,Nguyen,Create,Shipping Incoming,Completion,"Status:""Receive"", Receive:""2023/07/21 10:58:51...","Status:""Create"", Receive:"""", RECIEVE_USER_NAME:"""""
3,2G361K131,FMSV2023070056,2023/07/21 10:58:45,Nguyen,NaN,Basic,Create,"State:""South"", City:""HCM"", Customer:""FMSV"", Cu...",NaN


In [173]:
job_translog= {'RMA':'',
               'SN':'',
               'Receive':'',
               'Inspection':'',
               'Report Done':'',
               'Report Verified':'',
               'Quote Sent':'',
               'Confirmation':'',
               'Checked parts':'',
               'PO Created':'',
               'APAS Repair':'',
               'APAS Approved':'',
                
              }
# sn = df['S/N'][0]

job_translog['RMA'] = df['RMA#'][0]
job_translog['SN'] = df['S/N'][0]
for i in range(len(df)):
    # 1. Receive
    if df['Screen'][i]=='Shipping Incoming' and df['Type'][i]=='Completion':
        dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
#         job_translog.update({'Receive':[dtime.strftime('%Y-%m-%d'),df['User'][i]]})
        job_translog['Receive'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
    
    # 2. Inspection
    if df['Screen'][i]=='Inspection' and df['Type'][i]=='Completion':
        dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
        job_translog.update({'Inspection':[dtime.strftime('%Y-%m-%d'),df['User'][i]]})
    
    # 3. Report Done
    if  df['Updated Item(After)'][i] in ('Status Info:"Waiting for Next Process Available"',''):
        dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
        job_translog.update({'Report Done':[dtime.strftime('%Y-%m-%d'),df['User'][i]]})
    
    # 4. Report Verified
    if  df['Updated Item(After)'][i] in ('Status Info:"Awating Parts billing"',''):
        dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
        job_translog['Report Verified'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
    
    # 5. Quote Sent
    if  df['Updated Item(After)'][i] in ('Status Info:"Awating Contract billing"',''):
        dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
        job_translog['Quote Sent'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
    
    # 6. Check stock
    if  'Waiting for Parts' in df['Updated Item(After)'][i] or 'Parts on Order' in df['Updated Item(After)'][i]:
        dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
        job_translog['Checked parts'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
    # 7. PO Created
    if  'Parts on Order' in df['Updated Item(After)'][i]:
        dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
        job_translog['PO Created'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
    
    # 8. APAS repair
    if  df['Screen'][i]=='Parts Selection' and 'Awaiting Internal Approval' in df['Updated Item(After)'][i]:
        dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
        job_translog['APAS Repair'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
    
    # 9. APAS Approved
#     if 
        
    
job_translog

{'RMA': 'FMSV2023070056',
 'SN': '2G361K131',
 'Receive': ['2023-07-21', 'Nguyen'],
 'Inspection': ['2023-07-21', 'Nguyen'],
 'Report Done': '',
 'Report Verified': '',
 'Quote Sent': '',
 'Confirmation': '',
 'Checked parts': '',
 'PO Created': '',
 'APAS Repair': '',
 'APAS Approved': ''}

In [150]:
job_translog['Receive'][1]

'Nguyen'

In [118]:
dtime = dt.strptime(df['Time'][0],'%Y/%m/%d %H:%M:%S')
dtime.strftime('%Y-%m-%d')

'2023-08-14'

In [119]:
#
sn = df['S/N'][0]
rma = df['RMA#'][0]

In [121]:
print(rma)

FMSV2023070056
